# Library

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models import FastText

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from keras.utils.np_utils import to_categorical
from keras.models import load_model, Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing
### Load Data

In [2]:
df = pd.read_pickle('preprocessed_df.pkl')
df = pd.DataFrame(df[['detokenize','direktor']])
df.head()

,detokenize,direktor
0,the desperate hour lakewood salah cerita suara...,1
1,edisi males review singkat tonton libur dp des...,0
2,plot utama orang deserter pursuit buru wamil j...,0
3,film hereditary horror thrill midsommar gatau ...,0
4,batman manusiawi tarung nya sadis scene pursui...,0


In [3]:
reviews = df['detokenize'].astype(str)
label = pd.get_dummies(df['direktor']).values

# Variables

In [4]:
MAX_FEATURES = 1000
test_size_split = 0.2

# Split Data

In [5]:
x_train, x_test, y_train, y_test = (
    train_test_split(reviews, 
                     label, 
                     test_size=test_size_split, 
                     stratify = label, 
                     random_state=42)
)

print(f'Train\t| X shape: {x_train.shape}\tY shape: {y_train.shape}')
print(f'Test\t| X shape: {x_test.shape}\tY shape: {y_test.shape}')

Train	| X shape: (13797,)	Y shape: (13797, 3)
Test	| X shape: (3450,)	Y shape: (3450, 3)


# Feature Extraction

In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = MAX_FEATURES)
vectorizer.fit(x_train)

x_train_tfidf = vectorizer.fit_transform(x_train).todense()
x_test_tfidf = vectorizer.transform(x_test).todense()

# Feature Expansion

In [7]:
# Load trained fasttext model
model_exp = FastText.load('fasttext_news_tweet.model').wv

In [8]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 20) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [9]:
feature_ext = vectorizer.get_feature_names_out()

df_x_train = pd.DataFrame(x_train_tfidf, columns= feature_ext)
df_x_test = pd.DataFrame(x_test_tfidf, columns= feature_ext)

df_exp_x_train = feature_expansion(df_x_train, feature_ext)
df_exp_x_test = feature_expansion(df_x_test, feature_ext)

x_train = np.expand_dims(df_exp_x_train, 1)
x_test = np.expand_dims(df_exp_x_test, 1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 75.28it/s]


# Model RNN

In [10]:
def RNN(x_train_c, y_train_c, x_test_c, y_test_c):
    rnn = Sequential()
    rnn.add(SimpleRNN(units=256, activation='relu', recurrent_dropout=0.2, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=3, activation='softmax'))
    
    rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    rnn.build(x_train_c.shape)
    
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience=3, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint('rnn_tfidf_ft_direktor.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    
    history = rnn.fit(x_train_c, y_train_c, epochs= 10, batch_size=128, 
                      validation_data=(x_test_c, y_test_c), verbose = 1,
                      callbacks=[early_stopping, model_checkpoint])
    
    model = load_model('rnn_tfidf_ft_direktor.h5')
    y_pred = model.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)
   
    cm = confusion_matrix(y_test_cat, y_pred_cat)
    classreport = classification_report(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    precision = precision_score(y_test_cat, y_pred_cat, average="macro")
    accuracy = accuracy_score(y_test_cat, y_pred_cat)

    print(classreport)
    print("F1 Score : ", f1)
    print("Precision : ", precision)
    print("Recall : ", recall)
    print("Accuracy : ", accuracy)

    return [f1, precision, recall, accuracy, cm]

# Running

In [11]:
hasil = []

for i in range(1, 6):
    hasil.append([str(i)] + RNN(x_train, y_train, x_test, y_test))

Epoch 1/10
103/108 [===========================>..] - ETA: 0s - loss: 0.5151 - accuracy: 0.8465
Epoch 1: val_accuracy improved from -inf to 0.85536, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 9s 22ms/step - loss: 0.5123 - accuracy: 0.8466 - val_loss: 0.4197 - val_accuracy: 0.8554
Epoch 2/10
107/108 [============================>.] - ETA: 0s - loss: 0.3858 - accuracy: 0.8667
Epoch 2: val_accuracy improved from 0.85536 to 0.87217, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 1s 12ms/step - loss: 0.3849 - accuracy: 0.8672 - val_loss: 0.3841 - val_accuracy: 0.8722
Epoch 3/10
103/108 [===========================>..] - ETA: 0s - loss: 0.3304 - accuracy: 0.8877
Epoch 3: val_accuracy improved from 0.87217 to 0.87304, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 1s 11ms/step - loss: 0.3315 - accuracy: 0.8868 - val_loss: 0.3800 - val_accuracy: 0.8730
Epoch 4/10
103/108 [========

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
107/108 [============================>.] - ETA: 0s - loss: 0.4986 - accuracy: 0.8508
Epoch 1: val_accuracy improved from -inf to 0.85536, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 13s 36ms/step - loss: 0.4987 - accuracy: 0.8505 - val_loss: 0.4226 - val_accuracy: 0.8554
Epoch 2/10
108/108 [==============================] - ETA: 0s - loss: 0.3922 - accuracy: 0.8585
Epoch 2: val_accuracy improved from 0.85536 to 0.86667, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 3s 29ms/step - loss: 0.3922 - accuracy: 0.8585 - val_loss: 0.3938 - val_accuracy: 0.8667
Epoch 3/10
107/108 [============================>.] - ETA: 0s - loss: 0.3362 - accuracy: 0.8846
Epoch 3: val_accuracy improved from 0.86667 to 0.87652, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 4s 33ms/step - loss: 0.3359 - accuracy: 0.8848 - val_loss: 0.3754 - val_accuracy: 0.8765
Epoch 4/10
107/108 [=======

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
106/108 [============================>.] - ETA: 0s - loss: 0.5231 - accuracy: 0.8432
Epoch 1: val_accuracy improved from -inf to 0.85536, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 10s 45ms/step - loss: 0.5223 - accuracy: 0.8432 - val_loss: 0.4232 - val_accuracy: 0.8554
Epoch 2/10
108/108 [==============================] - ETA: 0s - loss: 0.3994 - accuracy: 0.8577
Epoch 2: val_accuracy improved from 0.85536 to 0.86957, saving model to rnn_tfidf_ft_direktor.h5
108/108 [==============================] - 4s 36ms/step - loss: 0.3994 - accuracy: 0.8577 - val_loss: 0.3912 - val_accuracy: 0.8696
Epoch 3/10
107/108 [============================>.] - ETA: 0s - loss: 0.3415 - accuracy: 0.8822
Epoch 3: val_accuracy did not improve from 0.86957
108/108 [==============================] - 3s 26ms/step - loss: 0.3409 - accuracy: 0.8824 - val_loss: 0.3788 - val_accuracy: 0.8696
Epoch 4/10
107/108 [============================>.] - ETA: 0s - loss: 0.2

# Save Output

In [12]:
df = pd.DataFrame(hasil, columns =['iterasi', 'f1', 'precision', 'recall', 'accuracy', 'cm'])
#df.to_excel('rnn_tfidf_ft_direktor.xlsx')
df.head()

,iterasi,f1,precision,recall,accuracy,cm
0,1,0.489499,0.605973,0.464047,0.874203,"[[4, 101, 12], [6, 2864, 81], [3, 231, 148]]"
1,2,0.452449,0.518384,0.431525,0.875362,"[[0, 108, 9], [0, 2901, 50], [0, 263, 119]]"
2,3,0.490420,0.758006,0.456775,0.879710,"[[5, 98, 14], [1, 2898, 52], [1, 249, 132]]"
3,4,0.451902,0.548522,0.427255,0.880000,"[[0, 108, 9], [0, 2925, 26], [0, 271, 111]]"
4,5,0.478040,0.774983,0.448038,0.879130,"[[3, 103, 11], [1, 2902, 48], [0, 254, 128]]"


In [13]:
print("Accuracy : %.2f" % (df['accuracy'].mean()*100))
print("F1-Score : %.2f" % (df['f1'].mean()*100))

Accuracy : 87.77
F1-Score : 47.25
